In [ ]:
import pandas as pd
import zipfile
import re
from sklearn.preprocessing import LabelEncoder
from smbox.utils import Logger
from smbox.optimise import Optimise
from smbox.smbox_config import smbox_params
from smbox.paramspace import rf_default_param_space, xgb_default_param_space

pd.set_option('display.max_columns', None)

zip_file_path = './playground-series-s3e17.zip' # example dataset
with zipfile.ZipFile(zip_file_path, 'r') as z:
    
    # Open the CSV file contained in the ZIP archive
    with z.open('train.csv') as f:
        # Read the CSV file into a pandas DataFrame
        df_train = pd.read_csv(f)
        
    with z.open('test.csv') as f:
        df_test = pd.read_csv(f)

In [ ]:
print(df_train.shape)
display(df_train.head(2))

In [ ]:
def preprocess_data(df_input, label_encode_col, one_hot_encode_col):
    """
    Preprocesses a given DataFrame by label encoding and one-hot encoding specified columns.
    
    This function:
    1. Removes the 'id' column if present.
    2. Removes or replaces forbidden characters from column names.
    3. Label encodes the specified column and drops the original column.
    4. One-hot encodes the specified column and concatenates the resultant columns to the DataFrame, then drops the original column.

    Parameters:
    - df_input (pd.DataFrame): The input DataFrame to preprocess.
    - label_encode_col (str): The column name to be label encoded.
    - one_hot_encode_col (str): The column name to be one-hot encoded.

    Returns:
    - pd.DataFrame: The preprocessed DataFrame with the specified columns encoded accordingly.
    """
    df = df_input.copy()
    
    df.drop('id', axis=1, inplace=True)
    
    # Remove or replace forbidden characters
    df.columns = [re.sub(r'[<>\[\]]', '', col) for col in df.columns]

    # Instantiate labelencoder object
    labelencoder = LabelEncoder()
    
    # Apply LabelEncoder on the specified column
    encoded_col_name = label_encode_col + '_encoded'
    df[encoded_col_name] = labelencoder.fit_transform(df[label_encode_col])
    
    # Drop the original label encoding column from the DataFrame
    df.drop(label_encode_col, axis=1, inplace=True)
    
    # Perform one-hot encoding on the specified column
    one_hot = pd.get_dummies(df[one_hot_encode_col], prefix=one_hot_encode_col)
    
    
    # Concatenate the original DataFrame with the one-hot encoded columns
    df = pd.concat([df, one_hot], axis=1)
    
    # Drop the original one-hot encoding column from the DataFrame
    df.drop(one_hot_encode_col, axis=1, inplace=True)
    
    return df

In [ ]:
# Use the function on df_train
# Drop the original label encoding column from the DataFrame

df_train_preprocessed = preprocess_data(df_train, 'Product ID', 'Type')
df_test_preprocessed = preprocess_data(df_test, 'Product ID', 'Type')

print(f'Columns: {list(df_train_preprocessed.columns)}')
target_name = 'Machine failure'
y_train = df_train_preprocessed[target_name]
X_train = df_train_preprocessed.drop(target_name, axis=1)
X_train.fillna(0, inplace=True)

In [ ]:
# Configuration settings for the experiment.
# This dictionary holds key details for the setup, including dataset details, algorithm choice, search strategy, etc.
# Some keys enhance the clarity of logs and outputs, ensuring reproducibility and transparency in experiments.
global config
config = {
    'dataset_source': 'openml',               # Dataset's source platform; 'openml' in this instance.
    'dataset': 38,                            # Unique identifier for the dataset on OpenML.
    'algorithm': 'rf',                        # Chosen algorithm: Random Forest (denoted as 'rf').
    'search_strategy': 'smbox',               # Optimization/search strategy, specified as 'smbox'.
    'search_strategy_config': smbox_params,   # Configuration specifics for 'smbox'. Assumes `smbox_params` is predefined.
    'wallclock': 600,                         # Maximum time allotted for the task (600 seconds or 10 minutes).
    'output_root': './output/'                # Directory for saving output/results.
}

logger = Logger()
logger.log(f'Experiment Config: {config}')

# Create a dictionary with training data. This format is needed for the SMBOX optimizer.
data = {"X_train": X_train, "y_train":y_train}

# Determine the default hyperparameter search space based on the algorithm specified in the configuration.
if config['algorithm'] == 'rf':
    # Use the default Random Forest hyperparameter search space.
    cfg_schema = rf_default_param_space
    
    # Customize the search space by setting an upper limit for the 'max_depth' parameter.
    cfg_schema['tune']['max_depth']['max'] = 250
elif config['algorithm'] == 'xgb':
    # Use the default XGBoost hyperparameter search space.
    cfg_schema = xgb_default_param_space
    
    # Update the 'scale_pos_weight' parameter, which helps in handling class imbalance.
    # Calculate the balance ratio between the two classes in the training data.
    classes = y_train.value_counts()
    class_0 = min(classes.index.values)
    class_1 = max(classes.index.values)
    balance_ratio = round(classes[class_0] / classes[class_1], 2)
    
    # Set the calculated balance ratio as the fixed value for 'scale_pos_weight'.
    cfg_schema['fix']['scale_pos_weight'] = balance_ratio

logger.log(f'-------------Starting SMBOX')
logger.log(f'Initial configuration schema: {cfg_schema}')

optimiser = Optimise(config, random_seed=42)
best_parameters, best_perf = optimiser.SMBOXOptimise(data, cfg_schema)

## Geneate predictions on the test set using the best found params

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

if config['algorithm'] == 'rf':
    model = RandomForestClassifier(**best_parameters, random_state=42)
elif config['algorithm'] == 'xgb':
    model = XGBClassifier(**best_parameters, random_state=42)

model.fit(X_train, y_train)

In [ ]:
df_test['Machine failure'] = model.predict_proba(df_test_preprocessed)[:, 1] 
df_submission = df_test[['id', 'Machine failure']]

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y_%m_%d_%H%M%S")
output_path = f"submission_{config['algorithm']}_{timestamp}.csv"

df_submission.to_csv(output_path, index=False)